![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/2.Clinical_Assertion_Model.ipynb)

# Clinical Assertion Status Model


The deep neural network architecture for assertion status detection in Spark NLP is based on a Bi-LSTM framework, and is a modified version of the architecture proposed by Federico Fancellu, Adam Lopez and Bonnie Webber ([Neural Networks For Negation Scope Detection](https://aclanthology.org/P16-1047.pdf)). Its goal is to classify the assertions made on given medical concepts as being present, absent, or possible in the patient, conditionally present in the patient under certain circumstances,
hypothetically present in the patient at some future point, and
mentioned in the patient report but associated with someoneelse.
In the proposed implementation, input units depend on the
target tokens (a named entity) and the neighboring words that
are explicitly encoded as a sequence using word embeddings.
Similar to paper mentioned above,  it is observed that that 95% of the scope tokens (neighboring words) fall in a window of 9 tokens to the left and 15
to the right of the target tokens in the same dataset. Therefore, the same window size was implemented and it following parameters were used: learning
rate 0.0012, dropout 0.05, batch size 64 and a maximum sentence length 250. The model has been implemented within
Spark NLP as an annotator called AssertionDLModel. After
training 20 epoch and measuring accuracy on the official test
set, this implementation exceeds the latest state-of-the-art
accuracy benchmarks as summarized as following table:

|Assertion Label|Spark NLP|Latest Best|
|-|-|-|
|Absent       |0.944 |0.937|
|Someone-else |0.904|0.869|
|Conditional  |0.441|0.422|
|Hypothetical |0.862|0.890|
|Possible     |0.680|0.630|
|Present      |0.953|0.957|
|micro F1     |0.939|0.934|


## Healthcare NLP for Data Scientists Course

If you are not familiar with the components in this notebook, you can check [Healthcare NLP for Data Scientists Udemy Course](https://www.udemy.com/course/healthcare-nlp-for-data-scientists/) and the [MOOC Notebooks](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/Spark_NLP_Udemy_MOOC/Healthcare_NLP) for each components.

**Colab Setup**

In [ ]:
import json
import os

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.4.1 spark-nlp==$PUBLIC_VERSION

# Installing NLU
! pip install --upgrade --q nlu==5.1.3 --no-dependencies

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [3]:
import json
import os

from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession

import nlu
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp_jsl.pretrained import InternalResourceDownloader

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 5.3.2
Spark NLP_JSL Version : 5.3.3


In [4]:
# if you want to start the session with custom params as in start function above
from pyspark.sql import SparkSession

def start(SECRET):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:"+PUBLIC_VERSION) \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+SECRET+"/spark-nlp-jsl-"+JSL_VERSION+".jar")

    return builder.getOrCreate()

#spark = start(SECRET)

# Clinical Assertion Models (with pretrained models)

|    | model_name              |Predicted Entities|
|---:|:------------------------|-|
|  1 | [assertion_dl](https://nlp.johnsnowlabs.com/2021/01/26/assertion_dl_en.html)            |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  2 | [assertion_dl_biobert](https://nlp.johnsnowlabs.com/2021/01/26/assertion_dl_biobert_en.html)    |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  3 | [assertion_dl_healthcare](https://nlp.johnsnowlabs.com/2020/09/23/assertion_dl_healthcare_en.html) |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  4 | [assertion_dl_large](https://nlp.johnsnowlabs.com/2020/05/21/assertion_dl_large_en.html)      |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  5 | [assertion_dl_radiology](https://nlp.johnsnowlabs.com/2021/03/18/assertion_dl_radiology_en.html)   |Confirmed, Suspected, Negative|
|  6 | [assertion_jsl](https://nlp.johnsnowlabs.com/2021/07/24/assertion_jsl_en.html)           |Present, Absent, Possible, Planned, Someoneelse, Past, Family, Hypotetical|
|  7 | [assertion_jsl_large](https://nlp.johnsnowlabs.com/2021/07/24/assertion_jsl_large_en.html)     |present, absent, possible, planned, someoneelse, past, hypothetical|
|  8 |  [assertion_ml](https://nlp.johnsnowlabs.com/2020/01/30/assertion_ml_en.html) |Hypothetical, Present, Absent, Possible, Conditional, Associated_with_someone_else|
|  9 | [assertion_dl_scope_L10R10](https://nlp.johnsnowlabs.com/2022/03/17/assertion_dl_scope_L10R10_en_3_0.html)| hypothetical, associated_with_someone_else, conditional, possible, absent, present|
| 10 | [assertion_dl_biobert_scope_L10R10](https://nlp.johnsnowlabs.com/2022/03/24/assertion_dl_biobert_scope_L10R10_en_2_4.html)| hypothetical, associated_with_someone_else, conditional, possible, absent, present|
| 11 | [assertion_jsl_augmented](https://nlp.johnsnowlabs.com/2022/09/15/assertion_jsl_augmented_en.html)| Present, Absent, Possible, Planned, Past, Family, Hypotetical, SomeoneElse|




### Pretrained `assertion_jsl_augmented` model

In [ ]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = MedicalNerModel.pretrained("ner_jsl", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")\
    #.setIncludeAllConfidenceScores(False)

ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\
    .setWhiteList(["SYMPTOM","VS_FINDING","DISEASE_SYNDROME_DISORDER","ADMISSION_DISCHARGE","PROCEDURE"])

# Assertion model trained on i2b2 (sampled from MIMIC) dataset
clinical_assertion = AssertionDLModel.pretrained("assertion_jsl_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")\
    .setEntityAssertionCaseSensitive(False)

nlpPipeline = Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter,
        clinical_assertion
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_jsl download started this may take some time.
[OK!]
assertion_jsl_augmented download started this may take some time.
[OK!]


In [ ]:
AssertionDLApproach().extractParamMap()

{Param(parent='AssertionDLApproach_283ec59a1319', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='AssertionDLApproach_283ec59a1319', name='label', doc='Column with one label per document'): 'label',
 Param(parent='AssertionDLApproach_283ec59a1319', name='batchSize', doc='Size for each batch in the optimization process'): 64,
 Param(parent='AssertionDLApproach_283ec59a1319', name='epochs', doc='Number of epochs for the optimization process'): 5,
 Param(parent='AssertionDLApproach_283ec59a1319', name='learningRate', doc='Learning rate for the optimization process'): 0.0012,
 Param(parent='AssertionDLApproach_283ec59a1319', name='dropout', doc='Dropout at the output of each layer'): 0.05,
 Param(parent='AssertionDLApproach_283ec59a1319', name='maxSentLen', doc='Max length for an input sentence.'): 250,
 Param(parent='AssertionDLApproach_283ec59a1319', name='includeConfidence', doc='whether to include confidence scores in a

In [ ]:
# we also have a LogReg based Assertion Model.
'''
clinical_assertion_ml = AssertionLogRegModel.pretrained("assertion_ml", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")
'''

In [ ]:
text = """
GENERAL: He is an elderly gentleman in no acute distress. He is sitting up in bed eating his breakfast. He is alert and oriented and answering questions appropriately.
HEENT: Sclerae showed mild arcus senilis in the right. Left was clear. Pupils are equally round and reactive to light. Extraocular movements are intact. Oropharynx is clear.
NECK: Supple. Trachea is midline. No jugular venous pressure distention is noted. No adenopathy in the cervical, supraclavicular, or axillary areas.
ABDOMEN: Soft and not tender. There may be some fullness in the left upper quadrant, although I do not appreciate a true spleen with inspiration.
EXTREMITIES: There is some edema, but no cyanosis and clubbing .
IMPRESSION: At this time is refractory anemia, which is transfusion dependent. He is on B12, iron, folic acid, and Procrit. There are no sign or symptom of blood loss and the previous esophagogastroduodenoscopy was negative. His creatinine was 1.
  My impression at this time is that he probably has an underlying myelodysplastic syndrome or bone marrow failure. His creatinine on this hospitalization was up slightly to 1.6 and this may contribute to his anemia.
  At this time, my recommendation for the patient is that he should undergo a bone marrow aspiration.
  I have discussed the procedure in detail which the patient. I have discussed the risks, benefits, and successes of that treatment and usefulness of the bone marrow and predicting his cause of refractory anemia and further therapeutic interventions, which might be beneficial to him.
  He is willing to proceed with the studies I have described to him. We will order an ultrasound of his abdomen because of the possible fullness of the spleen.
  As always, we greatly appreciate being able to participate in the care of your patient. We appreciate the consultation of the patient.
"""

In [ ]:
light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]
confidence=[]

for n,m in zip(light_result['ner_chunk'],light_result['assertion']):

    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    status.append(m.result)
    confidence.append(m.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, 'confidence':confidence})

df

,chunks,entities,assertion,confidence
0,distress,Symptom,Absent,0.9999
1,arcus senilis,Disease_Syndrome_Disorder,Past,1.0
2,jugular venous pressure distention,Symptom,Absent,1.0
3,adenopathy,Symptom,Absent,1.0
4,tender,Symptom,Absent,1.0
5,fullness,Symptom,Possible,1.0
6,edema,Symptom,Present,1.0
7,cyanosis,VS_Finding,Absent,1.0
8,clubbing,Symptom,Absent,1.0
9,anemia,Disease_Syndrome_Disorder,Hypothetical,0.9758


In [ ]:
from sparknlp_display import AssertionVisualizer

vis = AssertionVisualizer()

vis.display(light_result, 'ner_chunk', 'assertion')

In [ ]:
nlu.to_pretty_df(model,text,output_level='chunk').columns

Index(['assertion', 'assertion_confidence', 'document', 'entities_ner_chunk',
       'entities_ner_chunk_class', 'entities_ner_chunk_confidence',
       'entities_ner_chunk_origin_chunk', 'entities_ner_chunk_origin_sentence',
       'sentence_pragmatic', 'word_embedding_embeddings'],
      dtype='object')

In [ ]:
cols = [
     'entities_ner_chunk',
     'entities_ner_chunk_class',
     'assertion',
     'assertion_confidence']

df = nlu.to_pretty_df(model,text,output_level='chunk')[cols].reset_index(drop=True)
df


,entities_ner_chunk,entities_ner_chunk_class,assertion,assertion_confidence
0,distress,Symptom,Absent,0.9999
1,arcus senilis,Disease_Syndrome_Disorder,Past,1.0
2,jugular venous pressure distention,Symptom,Absent,1.0
3,adenopathy,Symptom,Absent,1.0
4,tender,Symptom,Absent,1.0
5,fullness,Symptom,Possible,1.0
6,edema,Symptom,Present,1.0
7,cyanosis,VS_Finding,Absent,1.0
8,clubbing,Symptom,Absent,1.0
9,anemia,Disease_Syndrome_Disorder,Hypothetical,0.9758


In [ ]:
# Downloading sample datasets.
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/mt_samples_10.csv

In [ ]:
mt_samples_df = spark.createDataFrame(pd.read_csv("/content/mt_samples_10.csv", sep=',', index_col=["index"]).reset_index())

mt_samples_df.printSchema()

root
 |-- index: long (nullable = true)
 |-- text: string (nullable = true)



In [ ]:
mt_samples_df.show()

+-----+--------------------+
|index|                text|
+-----+--------------------+
|    0|Sample Type / Med...|
|    1|Sample Type / Med...|
|    2|Sample Type / Med...|
|    3|Sample Type / Med...|
|    4|Sample Type / Med...|
|    5|Sample Type / Med...|
|    6|Sample Type / Med...|
|    7|Sample Type / Med...|
|    8|Sample Type / Med...|
|    9|Sample Type / Med...|
+-----+--------------------+



In [ ]:
result = model.transform(mt_samples_df)

In [ ]:
result.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|index|                text|            document|            sentence|               token|          embeddings|                 ner|           ner_chunk|           assertion|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    0|Sample Type / Med...|[{document, 0, 54...|[{document, 0, 24...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 68, 76, ...|[{assertion, 68, ...|
|    1|Sample Type / Med...|[{document, 0, 32...|[{document, 0, 26...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 68, 92, ...|[{assertion, 68, ...|
|    2|Sample Type / Med...|[{document, 0, 42...|[{document, 0, 14...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_

In [ ]:
result.select('sentence.result').take(1)

[Row(result=['Sample Type / Medical Specialty:\nHematology - Oncology\nSample Name:\nDischarge Summary - Mesothelioma - 1\nDescription:\nMesothelioma, pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.', '(Medical Transcription Sample Report)\nPRINCIPAL DIAGNOSIS:\nMesothelioma.', 'SECONDARY DIAGNOSES:\nPleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.', 'PROCEDURES', '1. On August 24, 2007, decortication of the lung with pleural biopsy and transpleural fluoroscopy.', '2. On August 20, 2007, thoracentesis.', '3. On August 31, 2007, Port-A-Cath placement.', 'HISTORY AND PHYSICAL:\nThe patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week.', 'She has had right-sided chest pain radiating to her back with fever starting yesterday.', 'She has a history of pericarditis and pericardectomy in May 2006 and developed cough with righ

In [ ]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.begin,
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata,
                                     result.assertion.result,
                                     result.assertion.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['3']['sentence']").alias("sent_id"),
              F.expr("cols['4']").alias("assertion"),
              F.expr("cols['5']['confidence']").alias("confidence") ).show(truncate=False)

+-------------------------+-----+---+-------------------------+-------+------------+----------+
|chunk                    |begin|end|ner_label                |sent_id|assertion   |confidence|
+-------------------------+-----+---+-------------------------+-------+------------+----------+
|Discharge                |68   |76 |Admission_Discharge      |0      |Past        |0.991     |
|pleural effusion         |132  |147|Disease_Syndrome_Disorder|0      |Present     |1.0       |
|anemia                   |171  |176|Disease_Syndrome_Disorder|0      |Family      |1.0       |
|ascites                  |179  |185|Disease_Syndrome_Disorder|0      |Hypothetical|0.9782    |
|esophageal reflux        |188  |204|Disease_Syndrome_Disorder|0      |Family      |1.0       |
|deep venous thrombosis   |222  |243|Disease_Syndrome_Disorder|0      |Family      |1.0       |
|Pleural effusion         |340  |355|Disease_Syndrome_Disorder|2      |Present     |1.0       |
|anemia                   |379  |384|Dis

### Pretrained `assertion_dl_radiology` model

In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetectorDLModel\
    .pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

# NER model for radiology
radiology_ner = MedicalNerModel.pretrained("ner_radiology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")\
    #.setIncludeAllConfidenceScores(False)

ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\
    .setWhiteList(["ImagingFindings"])

# Assertion model trained on radiology dataset
radiology_assertion = AssertionDLModel.pretrained("assertion_dl_radiology", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")

nlpPipeline = Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        radiology_ner,
        ner_converter,
        radiology_assertion
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
radiologyAssertion_model = nlpPipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_radiology download started this may take some time.
[OK!]
assertion_dl_radiology download started this may take some time.
[OK!]


In [ ]:
radiology_assertion.getClasses()

['Negative', 'Confirmed', 'Suspected']

In [ ]:
# A sample text from a radiology report

text = """No right-sided pleural effusion or pneumothorax is definitively seen and there are mildly displaced fractures of the left lateral 8th and likely 9th ribs."""

In [ ]:
data = spark.createDataFrame([[text]]).toDF("text")

In [ ]:
result = radiologyAssertion_model.transform(data)

In [ ]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.metadata,
                                     result.assertion.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label"),
              F.expr("cols['1']['sentence']").alias("sent_id"),
              F.expr("cols['2']").alias("assertion")).show(truncate=False)

+-------------------+---------------+-------+---------+
|chunk              |ner_label      |sent_id|assertion|
+-------------------+---------------+-------+---------+
|effusion           |ImagingFindings|0      |Negative |
|pneumothorax       |ImagingFindings|0      |Negative |
|displaced fractures|ImagingFindings|0      |Confirmed|
+-------------------+---------------+-------+---------+



## Writing a generic Assertion + NER function

In [ ]:
def get_base_pipeline (embeddings = 'embeddings_clinical'):

    documentAssembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

    # Sentence Detector annotator, processes various sentences per line
    sentenceDetector = SentenceDetector()\
        .setInputCols(["document"])\
        .setOutputCol("sentence")

    # Tokenizer splits words in a relevant format for NLP
    tokenizer = Tokenizer()\
        .setInputCols(["sentence"])\
        .setOutputCol("token")

    # Clinical word embeddings trained on PubMED dataset
    word_embeddings = WordEmbeddingsModel.pretrained(embeddings, "en", "clinical/models")\
        .setInputCols(["sentence", "token"])\
        .setOutputCol("embeddings")

    base_pipeline = Pipeline(
        stages=[
            documentAssembler,
            sentenceDetector,
            tokenizer,
            word_embeddings
    ])

    return base_pipeline



def get_clinical_assertion (embeddings, spark_df, nrows = 100, ner_model_name = 'ner_clinical', assertion_model_name="assertion_dl"):

  # NER model trained on i2b2 (sampled from MIMIC) dataset
    loaded_ner_model = MedicalNerModel.pretrained(ner_model_name, "en", "clinical/models") \
        .setInputCols(["sentence", "token", "embeddings"]) \
        .setOutputCol("ner")

    ner_converter = NerConverterInternal() \
        .setInputCols(["sentence", "token", "ner"]) \
        .setOutputCol("ner_chunk")

  # Assertion model trained on i2b2 (sampled from MIMIC) dataset
  # coming from sparknlp_jsl.annotator !!
    clinical_assertion = AssertionDLModel.pretrained(assertion_model_name, "en", "clinical/models") \
        .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
        .setOutputCol("assertion")


    base_model = get_base_pipeline (embeddings)

    nlpPipeline = Pipeline(
        stages=[
            base_model,
            loaded_ner_model,
            ner_converter,
            clinical_assertion
    ])

    empty_data = spark.createDataFrame([[""]]).toDF("text")

    model = nlpPipeline.fit(empty_data)

    result = model.transform(spark_df.limit(nrows))

    result = result.withColumn("id", F.monotonically_increasing_id())

    result_df = result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                                     result.ner_chunk.metadata,
                                                     result.assertion.result,
                                                     result.assertion.metadata)).alias("cols")) \
                      .select(F.expr("cols['0']").alias("chunk"),
                              F.expr("cols['1']['entity']").alias("ner_label"),
                              F.expr("cols['2']").alias("assertion"),
                              F.expr("cols['3']['confidence']").alias("confidence"))\
                      .filter("ner_label!='O'")

    return result_df

In [ ]:
embeddings = 'embeddings_clinical'

ner_model_name = 'ner_clinical_large'

nrows = 100

ner_df = get_clinical_assertion (embeddings, mt_samples_df, nrows, ner_model_name)

ner_df.show(30,truncate=50)

ner_clinical_large download started this may take some time.
[OK!]
assertion_dl download started this may take some time.
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
+----------------------------+---------+---------+----------+
|                       chunk|ner_label|assertion|confidence|
+----------------------------+---------+---------+----------+
|                Mesothelioma|  PROBLEM|  present|    0.9996|
|                Mesothelioma|  PROBLEM|  present|    0.9996|
|            pleural effusion|  PROBLEM|  present|    0.9998|
|         atrial fibrillation|  PROBLEM|  present|       1.0|
|                      anemia|  PROBLEM|  present|    0.9999|
|                     ascites|  PROBLEM|  present|    0.9999|
|           esophageal reflux|  PROBLEM|  present|    0.9999|
|      deep venous thrombosis|  PROBLEM|  present|    0.8533|
|                Mesothelioma|  PROBLEM|  present|    0.9992|
|            Pleural eff

In [ ]:
embeddings = 'embeddings_clinical'

ner_model_name = 'ner_posology'

nrows = 100

ner_df = get_clinical_assertion (embeddings, mt_samples_df, nrows, ner_model_name)

ner_df.show()

ner_posology download started this may take some time.
[OK!]
assertion_dl download started this may take some time.
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
+----------------+---------+------------+----------+
|           chunk|ner_label|   assertion|confidence|
+----------------+---------+------------+----------+
|        Coumadin|     DRUG|hypothetical|    0.8709|
|            1 mg| STRENGTH| conditional|    0.7772|
|           daily|FREQUENCY| conditional|    0.5086|
|      Amiodarone|     DRUG|hypothetical|    0.8589|
|          100 mg| STRENGTH|hypothetical|    0.6143|
|             p.o|    ROUTE|hypothetical|    0.7991|
|           daily|FREQUENCY|     present|    0.9074|
|        Coumadin|     DRUG|     present|    0.9999|
|         Lovenox|     DRUG|     present|    0.9994|
|           40 mg| STRENGTH|     present|    0.9982|
|  subcutaneously|    ROUTE|     present|    0.9302|
|    chemotherapy|     DRUG|    

In [ ]:
embeddings = 'embeddings_clinical'

ner_model_name = 'ner_posology_greedy'

entry_data = spark.createDataFrame([["The patient did not take a capsule of Advil."]]).toDF("text")

ner_df = get_clinical_assertion (embeddings, entry_data, nrows, ner_model_name)

ner_df.show()

ner_posology_greedy download started this may take some time.
[OK!]
assertion_dl download started this may take some time.
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
+----------------+---------+---------+----------+
|           chunk|ner_label|assertion|confidence|
+----------------+---------+---------+----------+
|capsule of Advil|     DRUG|   absent|    0.9855|
+----------------+---------+---------+----------+



In [ ]:
embeddings = 'embeddings_clinical'

ner_model_name = 'ner_clinical'

entry_data = spark.createDataFrame([["The patient has no fever"]]).toDF("text")

ner_df = get_clinical_assertion (embeddings, entry_data, nrows, ner_model_name)

ner_df.show()

ner_clinical download started this may take some time.
[OK!]
assertion_dl download started this may take some time.
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
+-----+---------+---------+----------+
|chunk|ner_label|assertion|confidence|
+-----+---------+---------+----------+
|fever|  PROBLEM|   absent|     0.998|
+-----+---------+---------+----------+



In [ ]:
def get_clinical_assertion_light (light_model, text):

  light_result = light_model.fullAnnotate(text)[0]

  chunks=[]
  entities=[]
  status=[]
  confidence=[]

  for n,m in zip(light_result['ner_chunk'],light_result['assertion']):

      chunks.append(n.result)
      entities.append(n.metadata['entity'])
      status.append(m.result)
      confidence.append(m.metadata['confidence'])

  df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status,'confidence':confidence})

  return df

In [ ]:
clinical_text = """
Patient with severe fever and sore throat.
He shows no stomach pain and he maintained on an epidural and PCA for pain control.
He also became short of breath with climbing a flight of stairs.
After CT, lung tumor located at the right lower lobe. Father with Alzheimer.
"""

light_model = LightPipeline(model)

# get_clinical_assertion_light (light_model, clinical_text)

cols = [
     'entities_ner_chunk',
     'entities_ner_chunk_class',
     'assertion',
     'assertion_confidence']

df = nlu.to_pretty_df(light_model,clinical_text, output_level='chunk')[cols]
df

,entities_ner_chunk,entities_ner_chunk_class,assertion,assertion_confidence
0,fever,VS_Finding,Present,1.0
0,sore throat,Symptom,Present,1.0
0,stomach pain,Symptom,Absent,1.0
0,pain,Symptom,Hypothetical,1.0
0,short of breath,Symptom,Present,1.0
0,climbing a flight of stairs,Symptom,Present,0.9434
0,Alzheimer,Disease_Syndrome_Disorder,Family,0.8136


# Replace Entity Labels

In [ ]:
# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = MedicalNerModel.pretrained("ner_clinical_large","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

# Assertion model trained on i2b2 (sampled from MIMIC) dataset
clinical_replaced_assertion = AssertionDLModel.pretrained("assertion_jsl_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("replaced_assertion") \
    .setIncludeConfidence(True) \
    .setReplaceLabels({"Present":"Exist",
                        "Absent": "None",
                       "Conditional": "Possible",
                       "Hypothetical": "Possible"})

nlpPipeline = Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter,
        clinical_replaced_assertion
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

ner_clinical_large download started this may take some time.
[OK!]
assertion_jsl_augmented download started this may take some time.
[OK!]


In [ ]:
text = """
Patient with severe fever and sore throat.
He shows no stomach pain and he maintained on an epidural and PCA for pain control.
He also became short of breath with climbing a flight of stairs.
After CT, lung tumor located at the right lower lobe. Father with Alzheimer.
"""

light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)

chunks=[]
entities=[]
confidence=[]
status=[]

for assertion_row in light_result[0]["replaced_assertion"]:
  chunk_id = assertion_row.metadata["chunk"]
  for chunk_row in light_result[0]["ner_chunk"]:
    if chunk_id == chunk_row.metadata["chunk"]:
        chunks.append(chunk_row.result)
        entities.append(chunk_row.metadata['entity'])
        status.append(assertion_row.result)
        confidence.append(assertion_row.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'replaced_assertion':status, 'confidence':confidence})

df     # "Present" replaced with "Exist", "Absent" replaces with "None", and "Hypotetical" replaced with "Possible"

,chunks,entities,replaced_assertion,confidence
0,severe fever,PROBLEM,Exist,1.0
1,sore throat,PROBLEM,Exist,1.0
2,stomach pain,PROBLEM,None,1.0
3,an epidural,TREATMENT,Exist,1.0
4,PCA,TREATMENT,Past,0.9978
5,pain control,PROBLEM,Possible,1.0
6,short of breath,PROBLEM,Exist,1.0
7,CT,TEST,Past,0.9963
8,lung tumor,PROBLEM,Exist,1.0
9,Alzheimer,PROBLEM,Family,0.8136


# Entity Type Constraints


You can effortlessly constrain assertions based on specific entity types using a convenient dictionary format: `{"entity": [assertion_label1, assertion_label2, .. assertion_labelN]}`. When an entity is not found in the dictionary, no constraints are applied, ensuring flexibility in your data processing. With the `setEntityAssertionCaseSensitive` parameter, you can control the case sensitivity for both entities and assertion labels. Unleash the full potential of your NLP model with these cutting-edge additions to the AssertionDLModel.

In [ ]:
# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = MedicalNerModel.pretrained("ner_clinical_large","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

# Assertion model trained on i2b2 (sampled from MIMIC) dataset
clinical_assertion = AssertionDLModel.pretrained("assertion_jsl_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")\
    .setEntityAssertionCaseSensitive(False)\
    .setEntityAssertion({
        "PROBLEM": ["hypothetical", "absent"],
        "treAtment": ["present"],
        "TEST": ["POssible"],
    })

nlpPipeline = Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter,
        clinical_assertion
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

ner_clinical_large download started this may take some time.
[OK!]
assertion_jsl_augmented download started this may take some time.
[OK!]


In [ ]:
text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting.
She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation.
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl,  creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27.
'''

In [ ]:
light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)

chunks=[]
entities=[]
status=[]
confidence=[]

for assertion_row in light_result[0]["assertion"]:
  chunk_id = assertion_row.metadata["chunk"]
  for chunk_row in light_result[0]["ner_chunk"]:
    if chunk_id == chunk_row.metadata["chunk"]:
        chunks.append(chunk_row.result)
        entities.append(chunk_row.metadata['entity'])
        status.append(assertion_row.result)
        confidence.append(assertion_row.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, 'confidence':confidence})

df

,chunks,entities,assertion,confidence
0,metformin,TREATMENT,Exist,0.5364
1,glipizide,TREATMENT,Exist,0.9993
2,dapagliflozin,TREATMENT,Exist,1.0
3,HTG,PROBLEM,Possible,1.0
4,Physical examination,TEST,Possible,0.943
5,tenderness,PROBLEM,None,1.0
6,guarding,PROBLEM,None,1.0
7,rigidity,PROBLEM,Possible,0.9999


# Assertion Filterer
AssertionFilterer will allow you to filter out the named entities by the list of acceptable assertion statuses by using method `setWhiteList()` or to exlude some entityes by using `.setBlackList()` method. This annotator would be quite handy if you want to set a white list / black list for the acceptable assertion statuses like present or conditional; or  put into black list if you dont want absent conditions get out of your pipeline.

In [ ]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

clinical_ner = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")\
    #.setIncludeAllConfidenceScores(False)

ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")

clinical_assertion = AssertionDLModel.pretrained("assertion_jsl", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")

assertion_filterer = AssertionFilterer()\
    .setInputCols("sentence","ner_chunk","assertion")\
    .setOutputCol("assertion_filtered")\
    .setCaseSensitive(False)\
    .setWhiteList(["Present"])
#or .setBlackList([["absent"]])

nlpPipeline = Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter,
        clinical_assertion,
        assertion_filterer
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
assertionFilter_model = nlpPipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical download started this may take some time.
[OK!]
assertion_jsl download started this may take some time.
[OK!]


In [ ]:
text = 'Patient has a headache for the last 2 weeks, needs to get a head CT, and appears anxious when she walks fast. Alopecia noted. She denies pain.'

light_model = LightPipeline(assertionFilter_model)
light_result = light_model.annotate(text)

light_result.keys()

dict_keys(['assertion_filtered', 'document', 'ner_chunk', 'assertion', 'token', 'ner', 'embeddings', 'sentence'])

In [ ]:
list(zip(light_result['ner_chunk'], light_result['assertion']))

[('a headache', 'Present'),
 ('a head CT', 'Hypothetical'),
 ('anxious', 'Possible'),
 ('Alopecia', 'Present'),
 ('pain', 'Absent')]

In [ ]:
assertion_filterer.getWhiteList()

['Present']

In [ ]:
chunks=[]
entities=[]
status=[]
confidence=[]

light_result = light_model.fullAnnotate(text)[0]

for m in light_result['assertion_filtered']:

    chunks.append(m.result)
    entities.append(m.metadata['entity'])
    status.append(m.metadata['assertion'])
    confidence.append(m.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, 'confidence':confidence})

df

,chunks,entities,assertion,confidence
0,a headache,PROBLEM,Present,0.97150004
1,Alopecia,PROBLEM,Present,0.9949


As you see, there is no "pain" chunk since it has "absent" assertion label.

# Oncological Assertion Models

<div align="center">

|    | model_name              |Predicted Entities|
|---:|:------------------------|-|
| 1 | [assertion_oncology_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_wip_en.html) | Medical_History, Family_History, Possible, Hypothetical_Or_Absent|
| 2 | [assertion_oncology_problem_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_problem_wip_en.html) |Present, Possible, Hypothetical, Absent, Family|
| 3 | [assertion_oncology_treatment_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_treatment_binary_wip_en.html) |Present, Planned, Past, Hypothetical, Absent|
| 3 | [assertion_oncology_treatment_wip]() |Present, Planned, Past, Hypothetical, Absent|
| 4 | [assertion_oncology_response_to_treatment_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_response_to_treatment_wip_en.html) |Present_Or_Past, Hypothetical_Or_Absent|
| 5 | [assertion_oncology_test_binary_wip](https://nlp.johnsnowlabs.com/2022/10/01/assertion_oncology_test_binary_wip_en.html) |Present_Or_Past, Hypothetical_Or_Absent|
| 6 | [assertion_oncology_smoking_status_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_smoking_status_wip_en.html) |Absent, Past, Present|
| 7 | [assertion_oncology_family_history_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_family_history_wip_en.html) |Family_History, Other|
| 8 | [assertion_oncology_demographic_binary_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_demographic_binary_wip_en.html) |Patient, Someone_Else|

</div>

In [ ]:
embeddings = 'embeddings_clinical'

ner_model_name = 'ner_oncology_wip'

assertion_model_name='assertion_oncology_wip'

nrows = 100

ner_df = get_clinical_assertion (embeddings, mt_samples_df, nrows, ner_model_name,assertion_model_name )

ner_df.show(truncate = False)

ner_oncology_wip download started this may take some time.
[OK!]
assertion_oncology_wip download started this may take some time.
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
+------------------------+--------------------+------------+----------+
|chunk                   |ner_label           |assertion   |confidence|
+------------------------+--------------------+------------+----------+
|Mesothelioma            |Cancer_Dx           |Present     |0.9885    |
|Mesothelioma            |Cancer_Dx           |Hypothetical|0.981     |
|August 24, 2007         |Date                |Past        |0.9726    |
|decortication           |Cancer_Surgery      |Past        |0.994     |
|lung                    |Site_Lung           |Past        |0.9453    |
|pleural                 |Site_Other_Body_Part|Past        |0.9624    |
|biopsy                  |Pathology_Test      |Past        |0.9979    |
|transpleural fluoroscopy|Imaging_Test  

# Voice of Patient Assertion Models

<div align="center">

|    | model_name              |Predicted Entities|
|---:|:------------------------|-|
| 1        | [assertion_vop_clinical](https://nlp.johnsnowlabs.com/2023/08/17/assertion_vop_clinical_en.html)     | Hypothetical_Or_Absent, Present_Or_Past, SomeoneElse |
| 2          | [assertion_vop_clinical_medium](https://nlp.johnsnowlabs.com/2023/08/17/assertion_vop_clinical_medium_en.html)       | Hypothetical_Or_Absent, Present_Or_Past, SomeoneElse |
| 3          | [assertion_vop_clinical_large](https://nlp.johnsnowlabs.com/2023/08/17/assertion_vop_clinical_large_en.html)       | Hypothetical_Or_Absent, Present_Or_Past, SomeoneElse |
|||


</div>

[Assertion status model](https://nlp.johnsnowlabs.com/2023/08/17/assertion_vop_clinical_en.html) used to predict if an NER chunk refers to a positive finding from the patient (Present_Or_Past), or if it refers to a family member or another person (SomeoneElse) or if it is mentioned but not as something present (Hypothetical_Or_Absent).

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_detector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare", "en", "clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel().pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

ner = MedicalNerModel.pretrained("ner_vop", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\
    .setBlackList(['DATETIME', 'DRUG', 'ROUTE', 'AGE', 'FREQUENCY', 'BODYPART', 'SUBSTANCEQUANTITY','FORM', 'ADMISSIONDISCHARGE', 'TESTRESULT',
                    'MEDICALDEVICE','CLINICALDEPT', 'GENDER',  'DURATION',
                   ])
assertion = AssertionDLModel.pretrained("assertion_vop_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")

pipeline = Pipeline(
    stages=[
        document_assembler,
        sentence_detector,
        tokenizer,
        word_embeddings,
        ner,
        ner_converter,
        assertion
])

data = spark.createDataFrame([["I was feeling anxiety honestly. Can it bring on tremors? It was right after my friend was diagnosed with diabetes."]]).toDF("text")

assertion_pipeline = pipeline.fit(data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_vop download started this may take some time.
[OK!]
assertion_vop_clinical download started this may take some time.
[OK!]


In [ ]:
assertion.getClasses()

['Hypothetical_Or_Absent', 'Present_Or_Past', 'SomeoneElse']

In [ ]:
sample_text = '''Hello, I am a 20-year-old woman who was diagnosed with hyperthyroidism around a month ago. For approximately four months, I've been experiencing symptoms such as feeling light-headed, battling poor digestion, dealing with anxiety attacks, depression, a sharp pain on my left side chest, an elevated heart rate, and a significant loss of weight. Due to these conditions, I was admitted to the hospital and just got discharged recently. During my hospital stay, a number of different tests were carried out by various physicians who initially struggled to pinpoint my actual medical condition. These tests included numerous blood tests, a brain MRI, an ultrasound scan, and an endoscopy. At long last, I was examined by a homeopathic doctor who finally diagnosed me with hyperthyroidism, indicating my TSH level was at a low 0.15 while my T3 and T4 levels were normal. Additionally, I was found to be deficient in vitamins B12 and D. Hence, I've been on a regimen of vitamin D supplements once a week and a daily dose of 1000 mcg of vitamin B12. I've been undergoing homeopathic treatment for the last 40 days and underwent a second test after a month which showed my TSH level increased to 0.5. While I'm noticing a slight improvement in my feelings of weakness and depression, over the last week, I've encountered two new challenges: difficulty breathing and a dramatically increased heart rate. I'm now at a crossroads where I am unsure if I should switch to allopathic treatment or continue with homeopathy. I understand that thyroid conditions take a while to improve, but I'm wondering if both treatments would require the same duration for recovery. Several of my acquaintances have recommended transitioning to allopathy and warn against taking risks, given the potential of developing severe complications. Please forgive any errors in my English and thank you for your understanding.'''

In [ ]:
from sparknlp_display import AssertionVisualizer

vis = AssertionVisualizer()

light_model = LightPipeline(assertion_pipeline)

light_result = light_model.fullAnnotate([sample_text])[0]

vis.display(light_result, 'ner_chunk', 'assertion')

# Social Determinant of Health Assertion Models

<div align="center">

|    | model_name              |Predicted Entities|
|---------------|----------------------|---|
| 1        | [assertion_sdoh_wip](https://nlp.johnsnowlabs.com/2023/08/13/assertion_sdoh_wip_en.html)     | `Present`, `Absent`, `Someone_Else`, `Past`, `Hypothetical`, `Possible` |

</div>


In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_detector = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "en")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

clinical_embeddings = WordEmbeddingsModel.pretrained('embeddings_clinical', "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

ner_model = MedicalNerModel.pretrained("ner_sdoh", "en", "clinical/models")\
    .setInputCols(["sentence", "token","embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(['sentence', 'token', 'ner'])\
    .setOutputCol('ner_chunk')\
    .setBlackList(['Age','Gender','Language','HEALTHCARE_INSTITUTION'])   # I dont need these assertion of entities

assertion = AssertionDLModel.pretrained("assertion_sdoh_wip", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")

pipeline = Pipeline(
    stages=[
        document_assembler,
        sentence_detector,
        tokenizer,
        clinical_embeddings,
        ner_model,
        ner_converter,
        assertion
])


empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_sdoh download started this may take some time.
[OK!]
assertion_sdoh_wip download started this may take some time.
[OK!]


In [ ]:
assertion.getClasses()

['Absent', 'Present', 'Someone_Else', 'Past', 'Hypothetical', 'Possible']

In [ ]:
sample_text= [
"""Smith works as a cleaning assistant and does not have access to health insurance or paid sick leave.
But she has generally housing problems. She lives in a apartment now.  She has long history of EtOH abuse, beginning in her teens.
She is aware she needs to attend Rehab Programs. She had DUI back in April and was due to be in court this week.
Her partner is an alcoholic and a drug abuser for the last 5 years.
She also mentioned feeling socially isolated and lack of a strong support system.
"""
]

In [ ]:
light_model = LightPipeline(model)
light_result = light_model.fullAnnotate(sample_text)

In [ ]:
light_result[0].keys()

dict_keys(['document', 'ner_chunk', 'assertion', 'token', 'ner', 'embeddings', 'sentence'])

In [ ]:
light_result[0]['ner_chunk']

[Annotation(chunk, 17, 34, cleaning assistant, {'chunk': '0', 'confidence': '0.76975', 'ner_source': 'ner_chunk', 'entity': 'Employment', 'sentence': '0'}, []),
 Annotation(chunk, 64, 79, health insurance, {'chunk': '1', 'confidence': '0.6325', 'ner_source': 'ner_chunk', 'entity': 'Insurance_Status', 'sentence': '0'}, []),
 Annotation(chunk, 156, 164, apartment, {'chunk': '2', 'confidence': '0.9575', 'ner_source': 'ner_chunk', 'entity': 'Housing', 'sentence': '2'}, []),
 Annotation(chunk, 196, 205, EtOH abuse, {'chunk': '3', 'confidence': '0.8286', 'ner_source': 'ner_chunk', 'entity': 'Alcohol', 'sentence': '3'}, []),
 Annotation(chunk, 265, 278, Rehab Programs, {'chunk': '4', 'confidence': '0.6292', 'ner_source': 'ner_chunk', 'entity': 'Access_To_Care', 'sentence': '4'}, []),
 Annotation(chunk, 289, 291, DUI, {'chunk': '5', 'confidence': '0.9603', 'ner_source': 'ner_chunk', 'entity': 'Legal_Issues', 'sentence': '5'}, []),
 Annotation(chunk, 363, 371, alcoholic, {'chunk': '6', 'confide

In [ ]:
chunks=[]
entities=[]
status=[]
confidence=[]

for assertion_row in light_result[0]["assertion"]:
  chunk_id = assertion_row.metadata["chunk"]
  for chunk_row in light_result[0]["ner_chunk"]:
    if chunk_id == chunk_row.metadata["chunk"]:
        chunks.append(chunk_row.result)
        entities.append(chunk_row.metadata['entity'])
        status.append(assertion_row.result)
        confidence.append(assertion_row.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, 'confidence':confidence})

df


,chunks,entities,assertion,confidence
0,cleaning assistant,Employment,Present,0.7926
1,health insurance,Insurance_Status,Absent,0.5072
2,apartment,Housing,Present,0.9956
3,EtOH abuse,Alcohol,Past,0.6054
4,Rehab Programs,Access_To_Care,Hypothetical,0.5861
5,DUI,Legal_Issues,Past,0.5037
6,alcoholic,Alcohol,Someone_Else,0.9868
7,drug abuser,Substance_Use,Someone_Else,0.9996
8,last 5 years,Substance_Duration,Someone_Else,0.9951
9,socially isolated,Social_Exclusion,Present,0.9673


In [ ]:
from sparknlp_display import AssertionVisualizer

vis = AssertionVisualizer()

vis.display(light_result[0], 'ner_chunk', 'assertion')

# AssertionChunkConverter

In some cases, there may be issues while creating the chunk column by using token indices and losing some data while training and testing the assertion status model if there are issues in these token indices. So we developed a new `AssertionChunkConverter` annotator that takes **begin and end indices of the chunks** as input and creates an extended chunk column with metadata that can be used for assertion status detection model training.

*NOTE*: Chunk begin and end indices in the assertion status model training dataframe can be populated using the new version of ALAB module.

In [5]:
data = spark.createDataFrame([["An angiography showed bleeding in two vessels off of the Minnie supplying the sigmoid that were succesfully embolized.", "Minnie", 57, 62],
     ["After discussing this with his PCP, Leon was clear that the patient had had recurrent DVTs and ultimately a PE and his PCP felt strongly that he required long-term anticoagulation ", "PCP", 31, 33]])\
     .toDF("text", "target", "char_begin", "char_end")

data.show()

+--------------------+------+----------+--------+
|                text|target|char_begin|char_end|
+--------------------+------+----------+--------+
|An angiography sh...|Minnie|        57|      62|
|After discussing ...|   PCP|        31|      33|
+--------------------+------+----------+--------+



In [6]:
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("tokens")

converter = AssertionChunkConverter() \
    .setInputCols("tokens")\
    .setChunkTextCol("target")\
    .setChunkBeginCol("char_begin")\
    .setChunkEndCol("char_end")\
    .setOutputTokenBeginCol("token_begin")\
    .setOutputTokenEndCol("token_end")\
    .setOutputCol("chunk")

pipeline = Pipeline().setStages([
    document_assembler,
    sentenceDetector,
    tokenizer,
    converter
])

results = pipeline.fit(data).transform(data)

In [7]:
results\
    .selectExpr(
        "target",
        "char_begin",
        "char_end",
        "token_begin",
        "token_end",
        "tokens[token_begin].result",
        "tokens[token_end].result",
        "target",
        "chunk")\
    .show(truncate=False)

+------+----------+--------+-----------+---------+--------------------------+------------------------+------+----------------------------------------------+
|target|char_begin|char_end|token_begin|token_end|tokens[token_begin].result|tokens[token_end].result|target|chunk                                         |
+------+----------+--------+-----------+---------+--------------------------+------------------------+------+----------------------------------------------+
|Minnie|57        |62      |10         |10       |Minnie                    |Minnie                  |Minnie|[{chunk, 57, 62, Minnie, {sentence -> 0}, []}]|
|PCP   |31        |33      |5          |5        |PCP                       |PCP                     |PCP   |[{chunk, 31, 33, PCP, {sentence -> 0}, []}]   |
+------+----------+--------+-----------+---------+--------------------------+------------------------+------+----------------------------------------------+



# AssertionMerger

AssertionMerger merges variety assertion columns coming from Assertion annotators such as AssertionDL and AssertionLogReg.
AssertionMerger can filter, prioritize, and merge assertion annotations by using proper parameters.

**Parameters:**

- `mergeOverlapping`: Whether to merge overlapping matched assertion annotations. Default: `True`
- `applyFilterBeforeMerge`: Whether to apply filtering before merging process. If `True`, filtering will be applied before merging; if `False`, filtering will be applied after merging process. Default: `False`.
- `blackList`: If defined, list of entities to ignore. The rest will be processed.
- `whiteList`: If defined, list of entities to process. The rest will be ignored. Do not include IOB prefix on labels.
- `caseSensitive`: Determines whether the definitions of the white listed and black listed entities are case sensitive. Default: `True`.
- `assertionsConfidence`: Pairs (assertion,confidenceThreshold) to filter assertions which have confidence lower than the confidence threshold.
- `orderingFeatures`: Specifies the ordering features to use for overlapping entities. Possible values include: 'begin', 'end', 'length', 'source', 'confidence'. Default: `['begin', 'length', 'source']`
- `electionStrategy`: Determines the strategy for selecting annotations. Annotations can be selected either sequentially based on their order (Sequential) or using a more diverse strategy (DiverseLonger). Currently, only Sequential and DiverseLonger options are available. Default: `Sequential`.
- `defaultConfidence` :  When the confidence value is included in the orderingFeatures and a given annotation does not have any confidence, this parameter determines the value to be used. The default value is `0`.
- `assertionSourcePrecedence`: Specifies the assertion sources to use for prioritizing overlapping annotations when the 'source' ordering feature is utilized. This parameter contains a comma-separated list of assertion sources that drive the prioritization. Annotations will be prioritized based on the order of the given string.
- `sortByBegin`: Whether to sort the annotations by begin at the end of the merge and filter process. Default: `False`.




In [ ]:
from pyspark.sql.types import StringType

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentence_detector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
ner_jsl = MedicalNerModel.pretrained("ner_jsl", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_jsl")\
    #.setIncludeAllConfidenceScores(False)

ner_jsl_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner_jsl"]) \
    .setOutputCol("ner_jsl_chunk")\
    .setWhiteList(["SYMPTOM","VS_FINDING","DISEASE_SYNDROME_DISORDER","ADMISSION_DISCHARGE","PROCEDURE"])

# Assertion model trained on i2b2 (sampled from MIMIC) dataset
assertion_jsl = AssertionDLModel.pretrained("assertion_jsl_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_jsl_chunk", "embeddings"]) \
    .setOutputCol("assertion_jsl")\
    .setEntityAssertionCaseSensitive(False)

ner_clinical = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_clinical")\
    #.setIncludeAllConfidenceScores(False)

ner_clinical_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner_clinical"]) \
    .setOutputCol("ner_clinical_chunk")\

# Assertion model trained on radiology dataset
assertion_dl = AssertionDLModel.pretrained("assertion_dl", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_clinical_chunk", "embeddings"]) \
    .setOutputCol("assertion_dl")

assertion_merger = AssertionMerger() \
    .setInputCols("assertion_jsl", "assertion_dl") \
    .setOutputCol("assertion_merger") \
    .setMergeOverlapping(True) \
    .setSelectionStrategy("sequential") \
    .setAssertionSourcePrecedence("assertion_dl, assertion_jsl") \
    .setCaseSensitive(False) \
    .setAssertionsConfidence({"past": 0.70}) \
    .setOrderingFeatures(["length", "source", "confidence"]) \
    .setDefaultConfidence(0.50)\
    #.setBlackList(["HYPothetical"])

pipeline = Pipeline(
    stages=[
        document_assembler,
        sentence_detector,
        tokenizer,
        word_embeddings,
        ner_jsl,
        ner_jsl_converter,
        assertion_jsl,
        ner_clinical,
        ner_clinical_converter,
        assertion_dl,
        assertion_merger
])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_jsl download started this may take some time.
[OK!]
assertion_jsl_augmented download started this may take some time.
[OK!]
ner_clinical download started this may take some time.
[OK!]
assertion_dl download started this may take some time.
[OK!]


In [ ]:
data = spark.createDataFrame([
"""Patient had a headache for the last 2 weeks, and appears anxious when she walks fast. No alopecia noted. She denies pain. Her father is paralyzed and it is a stressor for her. She got antidepressant. We prescribed sleeping pills for her current insomnia."""
], StringType()).toDF("text")


data = data.coalesce(1).withColumn("idx", F.monotonically_increasing_id())
results = pipeline.fit(data).transform(data)

In [ ]:
results.select("idx",F.explode(F.arrays_zip(results.assertion_merger.metadata,
                                            results.assertion_merger.begin,
                                            results.assertion_merger.end,
                                            results.assertion_merger.result)).alias("cols")) \
        .select("idx",F.expr("cols['0']['ner_chunk']").alias("ner_chunk"),
                F.expr("cols['1']").alias("begin"),
                F.expr("cols['2']").alias("end"),
                F.expr("cols['0']['ner_label']").alias("ner_label"),
                F.expr("cols['3']").alias("assertion"),
                F.expr("cols['0']['assertion_source']").alias("assertion_source"),
                F.expr("cols['0']['confidence']").alias("confidence"),
                ).sort("idx","begin").show(truncate=False)

+---+--------------+-----+---+---------+---------+----------------+----------+
|idx|ner_chunk     |begin|end|ner_label|assertion|assertion_source|confidence|
+---+--------------+-----+---+---------+---------+----------------+----------+
|0  |headache      |14   |21 |Symptom  |Past     |assertion_jsl   |0.9999    |
|0  |anxious       |57   |63 |PROBLEM  |present  |assertion_dl    |0.9392    |
|0  |alopecia      |89   |96 |PROBLEM  |absent   |assertion_dl    |0.9992    |
|0  |pain          |116  |119|PROBLEM  |absent   |assertion_dl    |0.9884    |
|0  |paralyzed     |136  |144|Symptom  |Family   |assertion_jsl   |0.9995    |
|0  |stressor      |158  |165|Symptom  |Family   |assertion_jsl   |1.0       |
|0  |antidepressant|184  |197|TREATMENT|present  |assertion_dl    |0.9628    |
|0  |sleeping pills|214  |227|TREATMENT|present  |assertion_dl    |0.998     |
|0  |insomnia      |245  |252|Symptom  |Past     |assertion_jsl   |0.9862    |
+---+--------------+-----+---+---------+---------+--

# Train a custom Assertion Model

In [ ]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/i2b2_assertion_sample_short.csv

In [ ]:
import pandas as pd

In [ ]:
assertion_df = spark.read.option("header", True).option("inferSchema", "True").csv("i2b2_assertion_sample_short.csv")

In [ ]:
assertion_df.show(3, truncate=100)

+-------------------------------------------------+-------------------+-------+-----+---+
|                                             text|             target|  label|start|end|
+-------------------------------------------------+-------------------+-------+-----+---+
|She has no history of liver disease , hepatitis .|      liver disease| absent|    5|  6|
|                         1. Undesired fertility .|undesired fertility|present|    1|  2|
|                            3) STATUS POST FALL .|               fall|present|    3|  3|
+-------------------------------------------------+-------------------+-------+-----+---+
only showing top 3 rows



In [ ]:
(training_data, test_data) = assertion_df.randomSplit([0.8, 0.2], seed = 100)
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 721
Test Dataset Count: 170


In [ ]:
training_data.groupBy('label').count().orderBy('count', ascending=False).show(truncate=False)

+-------+-----+
|label  |count|
+-------+-----+
|present|546  |
|absent |175  |
+-------+-----+



In [ ]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

token = Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")

chunk2doc = Doc2ChunkInternal()\
    .setInputCols(["document","token"])\
    .setOutputCol("chunk")\
    .setChunkCol("target")\
    .setStartCol("start")\
    .setStartColByTokenIndex(True)\
    .setFailOnMissing(False)\
    .setLowerCase(True)

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["document", "token"])\
    .setOutputCol("embeddings")

clinical_assertion_pipeline = Pipeline(
    stages = [
        document,
        token,
        chunk2doc,
        embeddings
])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


We will transform our test data with a pipeline consisting of same steps with the pipeline which contains AssertionDLApproach.
By doing this, we enable that test data will have same columns with training data in AssertionDLApproach. <br/>
The goal of this implementation is enabling the usage of `setTestDataset()` parameter in AssertionDLApproach.

In [ ]:
assertion_test_data = clinical_assertion_pipeline.fit(test_data).transform(test_data)

In [ ]:
assertion_test_data.columns

['text',
 'target',
 'label',
 'start',
 'end',
 'document',
 'token',
 'chunk',
 'embeddings']

We save the test data in parquet format to use in `AssertionDLApproach()`.

In [ ]:
assertion_test_data.write.mode('overwrite').parquet('i2b2_assertion_sample_test_data.parquet')

## Graph setup

In [ ]:
!pip install -q tensorflow==2.12.0
!pip install -q tensorflow-addons

We will use TFGraphBuilder annotator which can be used to create graphs in the model training pipeline.

TFGraphBuilder inspects the data and creates the proper graph if a suitable version of TensorFlow (<= 2.7 ) is available. The graph is stored in the defined folder and loaded by the approach.

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder= "./tf_graphs"

assertion_graph_builder = TFGraphBuilder()\
    .setModelName("assertion_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("label")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("assertion_graph.pb")\
    .setMaxSequenceLength(250)\
    .setHiddenUnitsNumber(25)

In [ ]:
'''
# ready to use tf_graph

!mkdir training_logs
!mkdir assertion_tf_graph

!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/tf_graphs/blstm_34_32_30_200_2.pb -P /content/assertion_tf_graph
'''

In [ ]:
'''
# create custom graph

from sparknlp_jsl.training import tf_graph
tf_graph.print_model_params("assertion_dl")

feat_size = 200
n_classes = 6

tf_graph.build("assertion_dl",
              build_params={"n_classes": n_classes},
              model_location= "./tf_graphs",
              model_filename="blstm_34_32_30_{}_{}.pb".format(feat_size, n_classes))
'''

**Setting the Scope Window (Target Area) Dynamically in Assertion Status Detection Models**


This parameter allows you to train the Assertion Status Models to focus on specific context windows when resolving the status of a NER chunk. The window is in format `[X,Y]` being `X` the number of tokens to consider on the left of the chunk, and `Y` the max number of tokens to consider on the right. Let’s take a look at what different windows mean:


*   By default, the window is `[-1,-1]` which means that the Assertion Status will look at all of the tokens in the sentence/document (up to a maximum of tokens set in `setMaxSentLen()` ).
*   `[0,0]` means “don’t pay attention to any token except the ner_chunk”, what basically is not considering any context for the Assertion resolution.
*   `[9,15]` is what empirically seems to be the best baseline, meaning that we look up to 9 tokens on the left and 15 on the right of the ner chunk to understand the context and resolve the status.


Check this [Scope Window Tuning Assertion Status Detection notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/2.1.Scope_window_tuning_assertion_status_detection.ipynb)  that illustrates the effect of the different windows and how to properly fine-tune your AssertionDLModels to get the best of them.

In our case, the best Scope Window is around [10,10]

In [ ]:
scope_window = [10,10]

assertionStatus = AssertionDLApproach()\
    .setLabelCol("label")\
    .setInputCols("document", "chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setBatchSize(64)\
    .setDropout(0.1)\
    .setLearningRate(0.001)\
    .setEpochs(20)\
    .setValidationSplit(0.2)\
    .setStartCol("start")\
    .setEndCol("end")\
    .setMaxSentLen(250)\
    .setIncludeConfidence(True)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('training_logs/')\
    .setGraphFolder(graph_folder)\
    .setGraphFile(f"{graph_folder}/assertion_graph.pb")\
    .setTestDataset(path="/content/i2b2_assertion_sample_test_data.parquet")\
    .setScopeWindow(scope_window)

'''
If .setTestDataset parameter is employed, raw test data cannot be fitted. .setTestDataset only works for dataframes which are correctly transformed
by a pipeline consisting of document, chunk, embeddings stages.
'''

In [ ]:
'''
assertionStatus = AssertionLogRegApproach()\
    .setLabelCol("label")\
    .setInputCols("document", "chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setMaxIter(100) # default: 26
'''

In [ ]:
clinical_assertion_pipeline = Pipeline(
    stages = [
        document,
        token,
        chunk2doc,
        embeddings,
        assertion_graph_builder,
        assertionStatus
])

In [ ]:
%%time

assertion_model = clinical_assertion_pipeline.fit(training_data)

TF Graph Builder configuration:
Model name: assertion_dl
Graph folder: ./tf_graphs
Graph file name: assertion_graph.pb
Build params: {'n_classes': 2, 'feat_size': 200, 'max_seq_len': 250, 'n_hidden': 25}


Instructions for updating:
non-resource variables are not supported in the long term


Device mapping: no known devices.


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Device mapping: no known devices.
assertion_dl graph exported to ./tf_graphs/assertion_graph.pb
CPU times: user 9.87 s, sys: 568 ms, total: 10.4 s
Wall time: 2min 5s


## Checking the results

Checking the results saved in the log file

In [ ]:
import os

log_files = os.listdir("./training_logs")
log_files

['AssertionDLApproach_426e45817583.log']

In [ ]:
with open("./training_logs/"+log_files[0]) as log_file:
    print(log_file.read())

Name of the selected graph: ./tf_graphs/assertion_graph.pb
Training started, trainExamples: 721


Epoch: 0 started, learning rate: 0.001, dataset size: 577
Done, 5.317838953 total training loss: 6.460061, avg training loss: 0.6460061, batches: 10
Quality on validation dataset (20.0%), validation examples = 144
time to finish evaluation: 1.04s
Total validation loss: 1.7350	Avg validation loss: 0.5783
label	 tp	 fp	 fn	 prec	 rec	 f1
present	 104	 30	 8	 0.7761194	 0.9285714	 0.8455284
absent	 2	 8	 30	 0.2	 0.0625	 0.095238104
tp: 106 fp: 38 fn: 38 labels: 2
Macro-average	 prec: 0.4880597, rec: 0.4955357, f1: 0.49176928
Micro-average	 prec: 0.7361111, rec: 0.7361111, f1: 0.7361111


Quality on test dataset: 
time to finish evaluation: 0.80s
Total test loss: 2.2206	Avg test loss: 0.7402
label	 tp	 fp	 fn	 prec	 rec	 f1
present	 111	 49	 6	 0.69375	 0.94871795	 0.80144405
absent	 4	 6	 49	 0.4	 0.0754717	 0.12698413
tp: 115 fp: 55 fn: 55 labels: 2
Macro-average	 prec: 0.546875, rec: 0.512

In [ ]:
preds = assertion_model.transform(test_data).select('label','assertion.result')

preds.show()

+-------+---------+
|  label|   result|
+-------+---------+
|present|[present]|
| absent|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
|present|[present]|
+-------+---------+
only showing top 20 rows



In [ ]:
preds_df = preds.toPandas()

In [ ]:
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])
preds_df

,label,result
0,present,present
1,absent,present
2,present,present
3,present,present
4,present,present
...,...,...
165,present,present
166,absent,absent
167,absent,absent
168,absent,absent


In [ ]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report( preds_df['label'], preds_df['result']))

              precision    recall  f1-score   support

      absent       0.79      0.72      0.75        53
     present       0.88      0.91      0.90       117

    accuracy                           0.85       170
   macro avg       0.83      0.82      0.82       170
weighted avg       0.85      0.85      0.85       170



In [ ]:
# save model
assertion_model.stages[-1].write().overwrite().save('assertion_custom_model')

## Load saved model

In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

clinical_ner = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical download started this may take some time.
[OK!]


In [ ]:
clinical_assertion = AssertionDLModel.load("assertion_custom_model") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")

nlpPipeline = Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter,
        clinical_assertion
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

In [ ]:
text = 'Patient has a headache for the last 2 weeks, needs to get a head CT, and appears anxious when she walks fast. No alopecia and pain noted'

light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)[0]

print(text)

chunks=[]
entities=[]
status=[]
confidence=[]

for n,m in zip(light_result['ner_chunk'],light_result['assertion']):

    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    status.append(m.result)
    confidence.append(m.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, 'confidence':confidence})

df

Patient has a headache for the last 2 weeks, needs to get a head CT, and appears anxious when she walks fast. No alopecia and pain noted


,chunks,entities,assertion,confidence
0,a headache,PROBLEM,present,0.9916
1,a head CT,TEST,present,0.9955
2,anxious,PROBLEM,present,0.7781
3,alopecia,PROBLEM,absent,0.7879
4,pain,PROBLEM,absent,0.7968


## Extra Informations

**ExceptionHandling**

A robust exception handling if the process is broken down due to corrupted inputs. If it is set as True, the annotator tries to process as usual and ff exception-causing data (e.g. corrupted record/ document) is passed to the annotator, an exception warning is emitted which has the exception message. Processing continues with the next one while the rest of the records within the same batch is parsed without interruption. This comes with a performance penalty. The default behaviour is False and will throw exception and break the process to inform users.


*Example*:
```python
clinical_assertion =  medical.AssertionDLModel.pretrained("assertion_jsl_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")\
    .setEntityAssertionCaseSensitive(False)
    .setDoExceptionHandling(True)
```